<a href="https://colab.research.google.com/github/eswens13/deep_learning/blob/master/cart_pole_gym.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
#remove " > /dev/null 2>&1" to see what is going on under the hood
!pip install gym pyvirtualdisplay > /dev/null 2>&1
!apt-get install -y xvfb python-opengl ffmpeg > /dev/null 2>&1

In [54]:
!pip install pyglet==1.3.2

In [0]:
import gym
from gym import logger as gymlogger
from gym.wrappers import Monitor
#gymlogger.set_level(40) #error only
import tensorflow as tf
import numpy as np
import random
import matplotlib
import matplotlib.pyplot as plt
%matplotlib inline
import math
import glob
import io
import base64
from IPython.display import HTML

from IPython import display as ipythondisplay

In [56]:
from pyvirtualdisplay import Display
display = Display(visible=0, size=(1400, 900))
display.start()
# You can just ignore the warning 'xdpyinfo was not found' for now.

xdpyinfo was not found, X start can not be checked! Please install xdpyinfo!


<Display cmd_param=['Xvfb', '-br', '-nolisten', 'tcp', '-screen', '0', '1400x900x24', ':1005'] cmd=['Xvfb', '-br', '-nolisten', 'tcp', '-screen', '0', '1400x900x24', ':1005'] oserror=None return_code=None stdout="None" stderr="None" timeout_happened=False>

In [0]:
# Utility functions to enable video recording of gym environment and displaying
# it. To enable video, just do:
#   "env = wrap_env(env)"

def show_video():
  mp4list = glob.glob('video/*.mp4')
  if len(mp4list) > 0:
    mp4 = mp4list[0]
    video = io.open(mp4, 'r+b').read()
    encoded = base64.b64encode(video)
    ipythondisplay.display(HTML(data='''<video alt="test" autoplay 
                loop controls style="height: 400px;">
                <source src="data:video/mp4;base64,{0}" type="video/mp4" />
             </video>'''.format(encoded.decode('ascii'))))
  else: 
    print("Could not find video")
    

def wrap_env(env):
  env = Monitor(env, './video', force=True)
  return env

In [0]:
# Where ENV_NAME is the environment that are using from Gym, eg 'CartPole-v0'
ENV_NAME = 'CartPole-v0'
env = wrap_env(gym.make(ENV_NAME)) #wrapping the env to render as a video

In [59]:
# Check out the action and observation spaces.
#
# According to the GitHub
# (https://github.com/openai/gym/blob/master/gym/envs/classic_control/cartpole.py),
# the discrete action space is defined as follows:
#
#     0 - Push the cart to the left
#     1 - Push the cart to the right
#
# and the continuous observation (state) space is defined as follows:
#
#     Num	Observation               Min             Max
#       0	Cart Position             -4.8            4.8
#       1	Cart Velocity             -Inf            Inf
#       2	Pole Angle                 -24 deg        24 deg
#       3	Pole Velocity At Tip      -Inf            Inf
#
print(env.action_space)
print(env.observation_space)

Discrete(2)
Box(4,)


In [60]:
# The video that is generated ends very quickly. After examining the code for
# the CartPole environment, I found that the threshold on the angle of the pole
# before the environment considers itself done is 12 degrees. Obviously, with a
# random selection policy, this will be hit very quickly.
for i_episode in range(20):
  env.reset()
  for t in range(100):
      env.render()
      action = env.action_space.sample() # Choose randomly between push left or
                                         # push right
      observation, reward, done, info = env.step(action)
      # Just take a look a the first observation:
      if (i_episode == 0) and (t == 0):
        print("Initial State:")
        print("\tCart Position: {}\n\tCart Velocity: {}".format( \
                  observation[0], observation[1]))
        print("\tPole Angle: {}\n\tPole Velocity: {}\n".format( \
                  observation[2], observation[3]))

      if done:
        if (i_episode == 0):
          print("Episode finished after {} timesteps\n".format(t+1))
          print("Final State:")
          print("\tCart Position: {}\n\tCart Velocity: {}".format( \
                  observation[0], observation[1]))
          print("\tPole Angle: {}\n\tPole Velocity: {}\n".format( \
                  observation[2], observation[3]))
        break;
env.close()
show_video()

Initial State:
	Cart Position: 0.01152971081264392
	Cart Velocity: 0.24208906720693307
	Pole Angle: 0.02920878008462663
	Pole Velocity: -0.3111074074918091

Episode finished after 20 timesteps

Final State:
	Cart Position: -0.0817165090993933
	Cart Velocity: -0.7507447671846365
	Pole Angle: 0.22781983875317552
	Pole Velocity: 1.567791629084882



In [0]:
def choose_action(x):
  """
  Defines the neural network architecture used for the reinforcement learning
  agent. This is a very simple architecture consisting of only a few dense
  layers followed by a softmax at the output.

  Inputs to the neural network are the current state observation as well as the
  last observation. Each observation consists of:
    Cart Position         [-4.8, 4.8]
    Cart Velocity         (-Inf, Inf)
    Pole Angle            [-24 deg, 24 deg]
    Pole Velocity At Tip  (-Inf, Inf)
  
  Arguments:
    x: The tensor of current and previous state observations.
  
  Returns:
    probs: The one hot encoded action vector (i.e. the decision for which action
           to take next).
  """
  y = tf.layers.dense(x, 8, activation='relu')
  y = tf.layers.dense(y, 8, activation='relu')
  y = tf.layers.dense(y, 8, activation='relu')
  y = tf.layers.dense(y, 2)
  probs = tf.nn.softmax(y)
  one_hot = tf.one_hot(tf.argmax(probs, axis=1), 2)
  return one_hot

def get_loss(obs_batch):
  """
  Computes the loss value for a batch of observations. This loss will be used to
  update the network weights after a given number of time steps have elapsed.

  Arguments:
    obs_batch:  A tensor of observations.
  
  Returns:
    loss:   A scalar loss value used to update the network weights.
  """
  # Currently, we just use the absolute value of the pole angle as the loss
  # value.
  angle = obs_batch[:,2]
  loss = tf.reduce_sum(tf.abs(angle))
  return loss

def get_update_op(lr, curr_step, max_steps, obs_batch):
  """
  Gets the update operation that applies gradients to the network weights.

  Arguments:
    lr:         The initial learning rate value (decay is applied in this
                function).
    curr_step:  The current time step in the simulation.
    max_steps:  The maximum time step in the simulation.
    obs_batch:  The tensor of most recent observations.
  
  Returns:
    update_op:  The update operation to be used to update the network weights.
  """
  # Apply learning rate decay.
  curr_lr = tf.train.polynomial_decay( \
                      lr, curr_step, max_steps, end_learning_rate=1e-5)
  
  # Use the Adam optimizer to backpropagate loss.
  optimizer = tf.train.AdamOptimizer(curr_lr)

  # Create the TF op for backprop.
  update_op = optimizer.minimize(get_loss(obs_batch))
  return update_op

In [0]:
with tf.Session() as sess:
  # Instantiate the computational graph.
  x = tf.placeholder(tf.float32, shape=(10, 4))
  probs = forward_pass(x)

  # Initialize the variables after they've been allocated.
  sess.run(tf.initialize_all_variables())

  # There is a warning that is output about bound methods not being able to be
  # transformed. According to this thread on GitHub, this is a benign warning.
  #   https://github.com/tensorflow/addons/issues/94
  action = sess.run(probs, feed_dict={x: inputs})
  print("Actions: {}".format(probs, one_hot))

total_steps = 0
max_steps = 10000
learning_rate = 1e-3
done = False

tf.reset_default_graph()

with tf.Session as sess:
  # Create variables, placeholders, ops, etc.
  curr_obs = tf.placeholder(tf.float32, shape=(1,4))
  prev_obs = tf.placeholder(tf.float32, shape=(1,4))
  curr_reward = tf.placeholder(tf.float32, shape(1,1))
  curr_state = tf.concat([curr_obs, prev_obs, curr_reward], axis=1)

  choose_action_op = choose_action(curr_state)

  curr_step_val = tf.placeholder(tf.int32, shape=(1,))
  max_step_val = tf.constant(max_steps, dtype=tf.int32)
  obs_batch = tf.placeholder(tf.float32, shape=(batch_size, 4))
  update_op = get_update_op(learning_rate, \
                            curr_step_val, \
                            max_step_val, \
                            obs_batch)

  # Initialize variables.
  sess.run(tf.initialize_all_variables())
  reward_signal = 0.0
  gamma = 0.75

  # Run the simulation
  while total_steps < max_steps:
    env.reset()
    prev_observation = [0.0, 0.0, 0.0, 0.0]
    curr_observation = [0.0, 0.0, 0.0, 0.0]

    while not done:
        env.render()

        # Get the next action.
        action = sess.run(choose_action_op, \
                          feed_dict={curr_obs: curr_observation, \
                                     prev_obs: prev_observation, \
                                     curr_reward: reward_signal})
        prev_obs = curr_obs
        curr_obs, reward, done, _ = env.step(action)
        reward_signal = (gamma * reward) + ((1.0 - gamma) * reward_signal)
        total_steps += 1

  env.close()
show_video()